# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\GabrielNichols\Desktop\dessoft\CDados-Projeto-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [31]:
filename = 'vaper.xlsx'

In [32]:
train = pd.read_excel(filename)
train.head(10)

,Treinamento,Relevancia
0,eu dando teto preto depois de 01 coisada no va...,0
1,minha vó fumando vaper kkkkkkkkkkkkkkkkkkkk,0
2,e eu que ganhei um vaper do meu patrão kkkkkk,0
3,lembrei agora que fiquei bebada segunda e diss...,0
4,aí q vontade de comprar um vaper novo,0
5,aparentemente eu tentei comprar um vaper bebad...,0
6,pensando em comprar um vaper vei,0
7,@negotalles depois que meu amigo tudo começou ...,0
8,minha proxima aquisição vai ser um vaper 🤓,0
9,"onde eu compro vaper aqui no rio de janeiro, q...",0


In [33]:
train['Treinamento'] = train['Treinamento'].str.replace('[!-.:?;]', '')
train.head(10)

,Treinamento,Relevancia
0,eu dando teto preto depois de 01 coisada no va...,0
1,minha vó fumando vaper kkkkkkkkkkkkkkkkkkkk,0
2,e eu que ganhei um vaper do meu patrão kkkkkk,0
3,lembrei agora que fiquei bebada segunda e diss...,0
4,aí q vontade de comprar um vaper novo,0
5,aparentemente eu tentei comprar um vaper bebad...,0
6,pensando em comprar um vaper vei,0
7,@negotalles depois que meu amigo tudo começou ...,0
8,minha proxima aquisição vai ser um vaper 🤓,0
9,onde eu compro vaper aqui no rio de janeiro qu...,0


In [34]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,"viciar en vaper ? serio ? nao to acreditando, ...",0
1,to há meses com um vaper que não acaba!!!!! to...,1
2,"em minha casinha, fumando meu vaper e na paz d...",0
3,@_biagrazyy ela usou um vaper e ainda me mostr...,1
4,@irafaelatavares eu sábado caindo bebeda.\nesp...,0


In [64]:
test['Teste'] = test['Teste'].str.replace('[!-.:?;]', '')
test

,Teste,Relevancia
0,viciar en vaper serio nao to acreditando me ...,0
1,to há meses com um vaper que não acaba to chocada,1
2,em minha casinha fumando meu vaper e na paz do...,0
3,@_biagrazyy ela usou um vaper e ainda me mostr...,1
4,@irafaelatavares eu sábado caindo bebeda\nespe...,0
...,...,...
195,e se eu comprar um vaper,0
196,eu guardo o vaper tão bem que as vezes vou pro...,0
197,pora só queria meu vaper de novo,0
198,@lunaisnice eu falo o vaper em si nunca usei,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Foram considerados relevantes os tweets que: 
    comparavam o produto com outros similarer;
    mencionavem os motivos do usuaria gostar ou não do produto;
    julgaram o produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [49]:
train_relevante = train.loc[train['Relevancia']==1,['Treinamento']]
train_relevante.head()

,Treinamento
12,@juliagab__ compra um pod vaper é horrível
14,não tá dando condições nem de comprar um vaper...
51,assistindo filme fumando vaper comendo brigade...
52,vaper n tem nada demais ms smp q eu to saindo ...
53,caralho deixei meu rim pra comprar o negócio d...


In [54]:
test_relevante = test.loc[test['Relevancia']==1,['Teste']]
test_relevante.head()

,Teste
1,to há meses com um vaper que não acaba to chocada
3,@_biagrazyy ela usou um vaper e ainda me mostr...
9,o brother me deu um vaper de uva pq disse que ...
13,a resistência do vaper dura um mês a minha foi...
14,entrei na onda vaper eletrônico tchau cigas


In [62]:
test_relevante['Teste'].str.split()

1      [to, há, meses, com, um, vaper, que, não, acab...
3      [@_biagrazyy, ela, usou, um, vaper, e, ainda, ...
9      [o, brother, me, deu, um, vaper, de, uva, pq, ...
13     [a, resistência, do, vaper, dura, um, mês, a, ...
14     [entrei, na, onda, vaper, eletrônico, tchau, c...
30     [eu, definitivamente, não, gosto, de, vaper, o...
31     [queria, tanto, um, vaper, grande, comprei, e,...
41     [só, com, o, vaper, já, gastei, 14900, esse, m...
51     [é, impressão, minha, ou, vaper, é, muito, mai...
58     [e, por, fim, me, acostumei, com, o, meu, vape...
78                [manter, meu, vaper, é, caro, viu, af]
85             [comprei, um, sabor, de, vaper, horrível]
93     [gente, ontem, eu, usei, um, vaper, 10, eu, pu...
96     [@phoriaboy, fumar, cigarro, é, estranho, não,...
97     [comprei, um, vaper, p, parar, de, fumar, ciga...
98     [eu, não, tô, indo, comprar, um, vaper, de, 60...
102    [a, pior, aquisição, que, eu, fiz, foi, desse,...
106    [ce, ta, doidoooo, valeu

In [63]:
train_relevante['Treinamento'].str.split()

12     [@juliagab__, compra, um, pod, vaper, é, horrí...
14     [não, tá, dando, condições, nem, de, comprar, ...
51     [assistindo, filme, fumando, vaper, comendo, b...
52     [vaper, n, tem, nada, demais, ms, smp, q, eu, ...
53     [caralho, deixei, meu, rim, pra, comprar, o, n...
62     [comprei, um, vaper, pra, parar, de, fumar, ag...
65     [gente, o, povo, compra, vaper, semi, novo/usa...
68            [n, tem, nada, nessa, porra, desse, vaper]
74                                  [vaper, é, tão, bom]
78     [assistindo, um, pod, citando, os, perigos, do...
82     [todo, mês, e, no, mínimo, 200, reais, de, jui...
88     [o, vaper, ta, desde, a, 2h, da, manhã, na, to...
92     [o, jovem, meu, deus, eu, acredito, tanto, na,...
95     [expectativa, comprar, o, vaper, pra, parar, f...
96     [eu, me, tornei, a, pessoa, q, eu, mais, criti...
98                          [vaper, é, brega, pacaralho]
113    [@gioest, aparentemente, tb, mexe, com, alguma...
124    [melhor, coisa, que, eu,

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**